In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging
import os

import numpy as np
import pandas as pd

import core.config as cconfig
import core.finance as cofinanc
import core.plotting as coplotti
import dataflow.model as dtfmod
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import helpers.hsql as hsql
import im_v2.ccxt.data.client as icdcl
import im_v2.common.universe as ivcu
import im_v2.im_lib_tasks as imvimlita
import oms as oms

In [ ]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

# Build the reconciliation config

In [ ]:
config_list = oms.build_reconciliation_configs()
config = config_list[0]
print(config)

# Specify data to load

In [ ]:
# The dict points to `system_log_dir` for different experiments.
system_log_path_dict = dict(config["system_log_path"].to_dict())

In [ ]:
configs = oms.load_config_from_pickle(system_log_path_dict)
# Diff configs.
diff_config = cconfig.build_config_diff_dataframe(
    {
        "prod_config": configs["prod"],
        "sim_config": configs["sim"],
    }
)
diff_config

In [ ]:
# This dict points to `system_log_dir/process_forecasts/portfolio` for different experiments.
data_type = "portfolio"
portfolio_path_dict = oms.get_system_log_paths(system_log_path_dict, data_type)
portfolio_path_dict

In [ ]:
# This dict points to `system_log_dir/dag/node_io/node_io.data` for different experiments.
data_type = "dag"
dag_path_dict = oms.get_system_log_paths(system_log_path_dict, data_type)
dag_path_dict

In [ ]:
# TODO(gp): Load the TCA data for crypto.
if config["meta"]["run_tca"]:
    tca_csv = os.path.join(root_dir, date_str, "tca/sau1_tca.csv")
    hdbg.dassert_file_exists(tca_csv)

In [ ]:
date_str = config["meta"]["date_str"]
# TODO(gp): @Grisha infer this from the data from prod Portfolio df, but allow to overwrite.
start_timestamp = pd.Timestamp(date_str + " 08:45:00", tz="America/New_York")
_LOG.info("start_timestamp=%s", start_timestamp)
end_timestamp = pd.Timestamp(date_str + " 10:40:00", tz="America/New_York")
_LOG.info("end_timestamp=%s", end_timestamp)

# Data delay analysis

In [ ]:
# Get the real-time `ImClient`.
# TODO(Grisha): ideally we should get the values from the config.
resample_1min = False
env_file = imvimlita.get_db_env_path("dev")
connection_params = hsql.get_connection_info_from_env_file(env_file)
db_connection = hsql.get_connection(*connection_params)
table_name = "ccxt_ohlcv_futures"
#
im_client = icdcl.CcxtSqlRealTimeImClient(
    resample_1min, db_connection, table_name
)

In [ ]:
# Get the universe.
# TODO(Grisha): get the version from the config.
vendor = "CCXT"
mode = "trade"
version = "v7.1"
as_full_symbol = True
full_symbols = ivcu.get_vendor_universe(
    vendor,
    mode,
    version=version,
    as_full_symbol=as_full_symbol,
)
full_symbols

In [ ]:
# Load the data for the reconciliation date.
# `ImClient` operates in UTC timezone.
start_ts = pd.Timestamp(date_str, tz="UTC")
end_ts = start_ts + pd.Timedelta(days=1)
columns = None
filter_data_mode = "assert"
df = im_client.read_data(
    full_symbols, start_ts, end_ts, columns, filter_data_mode
)
hpandas.df_to_str(df, num_rows=5, log_level=logging.INFO)

In [ ]:
# TODO(Grisha): move to a lib.
# Compute delay in seconds.
df["delta"] = (df["knowledge_timestamp"] - df.index).dt.total_seconds()
# Plot the delay over assets with the errors bars.
delta_per_asset = df.groupby(by=["full_symbol"])["delta"]
minimums = delta_per_asset.min()
maximums = delta_per_asset.max()
means = delta_per_asset.mean()
errors = [means - minimums, maximums - means]
# TODO(Grisha): sort by maximum delay.
means.plot(
    kind="bar", 
    yerr=errors,
    title="DB delay in seconds per asset"
)

# Compare DAG io

In [ ]:
# Get DAG node names.
dag_node_names = oms.get_dag_node_names(dag_path_dict["prod"])
dag_node_names

In [ ]:
# Get timestamps for the last DAG node.
dag_node_timestamps = oms.get_dag_node_timestamps(
    dag_path_dict["prod"], dag_node_names[-1], as_timestamp=True
)
dag_node_timestamps

In [ ]:
# Load DAG output for different experiments.
dag_df_dict = {}
for name, path in dag_path_dict.items():
    # Get DAG node names for every experiment.
    dag_nodes = oms.get_dag_node_names(path)
    # Get timestamps for the last node.
    dag_node_ts = oms.get_dag_node_timestamps(
        path, dag_nodes[-1], as_timestamp=True
    )
    # Get DAG output for the last node and the last timestamp.
    timestamp = dag_node_ts[-1]
    print(timestamp)
    dag_df_dict[name] = oms.get_dag_node_output(
        path, dag_nodes[-1], timestamp
    )
#hpandas.df_to_str(dag_df_dict["prod"], num_rows=5, log_level=logging.INFO)

In [ ]:
dag_df_dict["sim"]["vwap.ret_0.vol_adj_2_hat"]
#dag_df_dict["sim"][]

In [ ]:
col_name = "vwap.ret_0.vol_adj_2_hat_pct_change"

# Compute percentage difference.
compare_visually_dataframes_kwargs = {
    "diff_mode": "pct_change",
    "background_gradient": False,
}
diff_df = hpandas.compare_multiindex_dfs(
    dag_df_dict["prod"],
    dag_df_dict["sim"],
    compare_visually_dataframes_kwargs=compare_visually_dataframes_kwargs,
)
# Remove the sign and NaNs.
diff_df = diff_df.replace([np.inf, -np.inf], np.nan).abs()
# Check that data is the same.
diff_df[col_name].tail(1)

In [ ]:
dag_df_dict["prod"]

In [ ]:
col_name = "vwap.ret_0.vol_adj_2_hat"

#diff_df = (dag_df_dict["prod"][col_name].tail(1) - dag_df_dict["sim"][col_name].tail(1)).round(2)
pd.concat([dag_df_dict["prod"][col_name].tail(1), dag_df_dict["sim"][col_name].tail(1), diff_df])

In [ ]:
diff_df

In [ ]:
# Plot diffs over time.
diff_df.max(axis=1).plot()

In [ ]:
# Plot diffs over columns.
diff_df.max(axis=0).unstack().max(axis=1).plot(kind="bar")

In [ ]:
# Plot diffs over assets.
diff_df.max(axis=0).unstack().max(axis=0).plot(kind="bar")

In [ ]:
# Compute correlations.
prod_sim_dag_corr = dtfmod.compute_correlations(
    dag_df_dict["prod"],
    dag_df_dict["sim"],
)
hpandas.df_to_str(
    prod_sim_dag_corr.min(),
    num_rows=None,
    precision=3,
    log_level=logging.INFO,
)

In [ ]:
# Make sure they are exactly the same.
(dag_df_dict["prod"] - dag_df_dict["sim"]).abs().max().max()

# Compute research portfolio equivalent

In [ ]:
fep = dtfmod.ForecastEvaluatorFromPrices(
    **config["research_forecast_evaluator_from_prices"]["init"]
)
annotate_forecasts_kwargs = config["research_forecast_evaluator_from_prices"][
    "annotate_forecasts_kwargs"
].to_dict()
research_portfolio_df, research_portfolio_stats_df = fep.annotate_forecasts(
    dag_df_dict["prod"],
    **annotate_forecasts_kwargs,
    compute_extended_stats=True,
)
# TODO(gp): Move it to annotate_forecasts?
research_portfolio_df = research_portfolio_df.sort_index(axis=1)
# Align index with prod and sim portfolios.
research_portfolio_df = research_portfolio_df.loc[start_timestamp:end_timestamp]
research_portfolio_stats_df = research_portfolio_stats_df.loc[
    start_timestamp:end_timestamp
]
#
hpandas.df_to_str(research_portfolio_stats_df, num_rows=5, log_level=logging.INFO)

# Load logged portfolios

In [ ]:
portfolio_config = cconfig.Config.from_dict(
    {
        "start_timestamp": start_timestamp,
        "end_timestamp": end_timestamp,
        "freq": config["meta"]["bar_duration"],
        "normalize_bar_times": True,
    }
)
portfolio_config

In [ ]:
portfolio_dfs, portfolio_stats_dfs = oms.load_portfolio_dfs(
    portfolio_path_dict,
    portfolio_config,
)
# Add research portfolio.
portfolio_dfs["research"] = research_portfolio_df
#
hpandas.df_to_str(portfolio_dfs["prod"], num_rows=5, log_level=logging.INFO)

In [ ]:
# Add research df and combine into a single df.
portfolio_stats_dfs["research"] = research_portfolio_stats_df
portfolio_stats_df = pd.concat(portfolio_stats_dfs, axis=1)
#
hpandas.df_to_str(portfolio_stats_df, num_rows=5, log_level=logging.INFO)

In [ ]:
bars_to_burn = 1
coplotti.plot_portfolio_stats(portfolio_stats_df.iloc[bars_to_burn:])

In [ ]:
stats_computer = dtfmod.StatsComputer()
stats_sxs, _ = stats_computer.compute_portfolio_stats(
    portfolio_stats_df.iloc[bars_to_burn:], config["meta"]["bar_duration"]
)
display(stats_sxs)

# Compare pairwise portfolio correlations

In [ ]:
dtfmod.compute_correlations(
    research_portfolio_df,
    portfolio_dfs["prod"],
    allow_unequal_indices=True,
    allow_unequal_columns=True,
)

In [ ]:
dtfmod.compute_correlations(
    portfolio_dfs["prod"],
    portfolio_dfs["sim"],
    allow_unequal_indices=False,
    allow_unequal_columns=False,
)

In [ ]:
dtfmod.compute_correlations(
    research_portfolio_df,
    portfolio_dfs["sim"],
    allow_unequal_indices=True,
    allow_unequal_columns=True,
)

In [ ]:
if config["meta"]["run_tca"]:
    tca = cofinanc.load_and_normalize_tca_csv(tca_csv)
    tca = cofinanc.compute_tca_price_annotations(tca, True)
    tca = cofinanc.pivot_and_accumulate_holdings(tca, "")